## AULA 05 - Exercício

Baseado no que vimos em aula até este momento e utilizando a base de dados disponibilizada abaixo, configure o dbt em sua máquina e crie um modelo que crie uma view contendo as seguintes informações:

- Cidade
- Combustível
- Preco
- Data Coleta

Além disso para cada registro gere uma chave única utilizando o método de surrogate_key 

## Criação do Dataset Inicial

In [0]:
import urllib.request
urllib.request.urlretrieve("https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/dsan/2023/precos-gasolina-etanol-08.csv","/tmp/precos-gasolina-etanol-08.csv")
dbutils.fs.mkdirs("dbfs:/aula-04-datalake/bronze")
dbutils.fs.mv("file:/tmp/precos-gasolina-etanol-08.csv","dbfs:/aula-04-datalake/bronze/precos-gasolina-etanol-08.csv")

Out[1]: True

In [0]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Dados Combustível Preparação").getOrCreate()
)

In [0]:
endereco_servidor = "isabelle.db.elephantsql.com"
porta_servidor = "5432" # Neste caso utilizamos a porta padrão do PostgreSQL que é a 5432
nome_database = "kchfcuyp"

jdbc_connection_string = f"jdbc:postgresql://{endereco_servidor}:{porta_servidor}/{nome_database}"

print(f"Nossa connection String é: {jdbc_connection_string}")

Nossa connection String é: jdbc:postgresql://isabelle.db.elephantsql.com:5432/kchfcuyp


In [0]:
df_dados = spark.read.option("header", "true").option("delimiter", ";").csv("dbfs:/aula-04-datalake/bronze/precos-gasolina-etanol-08.csv")

(
    df_dados.write.format("jdbc")
        .option("driver", "org.postgresql.Driver")
        .option("url", jdbc_connection_string)
        .option("user", "kchfcuyp")
        .option("password", "lCHnZ5WMjhcPAyYPne40VozNp5VX1OdD")
        .option("dbtable", "preco_combustiveis")
        .save()
 )

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-704301768898452>:4
      1 df_dados = spark.read.option("header", "true").option("delimiter", ";").csv("dbfs:/aula-04-datalake/bronze/precos-gasolina-etanol-08.csv")
      3 (
----> 4     df_dados.write.format("jdbc")
      5         .option("driver", "org.postgresql.Driver")
      6         .option("url", jdbc_connection_string)
      7         .option("user", "kchfcuyp")
      8         .option("password", "lCHnZ5WMjhcPAyYPne40VozNp5VX1OdD")
      9         .option("dbtable", "preco_combustiveis")
     10         .save()
     11  )

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_nam